In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12, 12))

import requests
import random

In [3]:
import time

class Timer(object):
    def __init__(self, verbose=False):
        self.verbose = verbose

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.secs = self.end - self.start
        self.msecs = self.secs * 1000  # millisecs
        if self.verbose:
            print( 'elapsed time: %f ms' % self.msecs)

In [5]:
filename = 'CAMHDA301-20160724T030000Z.mov'
# local_filename = '/v1/berna/data/' + filename
# nginx_filename = '/v1/nginx_data/' + filename
ci_filename = '/v1/org/oceanobservatories/rawdata/files/RS03ASHS/PN03B/06-CAMHDA301/2016/07/24/' + filename
frame_number = 7000

num_frames = 10

def random_frames( n_frames ):
    random.seed(5)   # I want this to be _non_-random
    return random.sample(range(25000), n_frames )

noncached_port = 9080
cached_port = 7080
    

In [8]:
def get_frame( url, frame_number ):
    url = url + "/frame/%d" % i
    r = requests.get(url)
    ## TODO: Validation here
    
def get_local( port, frame_number ):
    get_frame( ("http://localhost:%d" % port) + local_filename, frame_number )
    
def get_nginx( port, frame_number ):
    get_frame( ("http://localhost:%d" % port) + nginx_filename, frame_number )
    
def get_ci( port, frame_number ):
    get_frame( ("http://localhost:%d" % port) + ci_filename, frame_number )


# Non-cached Lazycache operating on local data

In [156]:
times = []
for i in random_frames(num_frames):
    with Timer() as t:
        get_local( noncached_port, i )
    times.append(t)

msec = [t.msecs for t in times]
print("Local, noncached: %f ms per frame" % np.mean(msec))

Local, noncached: 619.247842 ms per frame


# Non-cached Lazycache operating through local HTTP server

In [157]:
times = []
for i in random_frames(num_frames):
    with Timer() as t:
        get_nginx( noncached_port, i )
    
    times.append(t)

msec = [t.msecs for t in times]
print("Nginx, noncached: %f ms per frame" % np.mean(msec))

Local, noncached: 621.354747 ms per frame


# Non-cached Lazycache contacting CI directly

In [163]:
times = []
for i in random_frames(num_frames):
    with Timer() as t:
        get_ci( noncached_port, i )
    
    times.append(t)

msec = [t.msecs for t in times]
print("CI, noncached: %f ms per frame" % np.mean(msec))

CI, noncached: 715.747571 ms per frame



# Cached operating on local data

In [160]:
times = []
for i in random_frames(num_frames):
    with Timer() as t:
        get_local( cached_port, i )
    
    times.append(t)

msec = [t.msecs for t in times]
print("Local, cached: %f ms per frame" % np.mean(msec))

Local, cached: 14.524841 ms per frame


In [161]:
times = []
for i in random_frames(num_frames):
    with Timer() as t:
        get_nginx( cached_port, i )
    
    times.append(t)

msec = [t.msecs for t in times]
print("Nginx, cached: %f ms per frame" % np.mean(msec))

Nginx, cached: 14.533353 ms per frame


In [162]:
times = []
for i in random_frames(num_frames):
    with Timer() as t:
        get_ci( cached_port, i )
    
    times.append(t)

msec = [t.msecs for t in times]
print("CI, noncached: %f ms per frame" % np.mean(msec))

CI, noncached: 15.593529 ms per frame


# Google App Engine instance of Lazycache

In [6]:
def gae_ci( frame_number ):
    get_frame( "https://camhd-app-dev.appspot.com" + ci_filename, frame_number )

In [20]:
times = []
for i in random_frames(num_frames):
    with Timer() as t:
        gae_ci( i )
    
    times.append(t)

msec = [t.msecs for t in times]
print("GAE, cached: %f ms per frame" % np.mean(msec))

GAE, cached: 147.112799 ms per frame
